## This notebook is devoted for arbitrage opportunity from fat finger event

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import yfinance as yf
from sklearn.cluster import KMeans
from colorama import Fore, Style
import seaborn as sns
from sklearn.metrics import r2_score
sns.set(style='darkgrid')
import plotly.graph_objects as go
import pickle

import sys
sys.path.append('../src') 
from stock_strategy import stock_strategy, stock_regression
from chinese_index_etf_strategy import etf_strategy, etf_regression

from scipy.stats import norm
from scipy.optimize import minimize
from scipy.optimize import fsolve

from kneed import KneeLocator

### Near ATM and long-dated option?

Validated using call-put parity, these "fat finger error" carries much less profitability than originally thought.

In [2]:
def option_symbol(
    strike: float,
    # expiration: datetime = datetime.today() + relativedelta(days=(4 - datetime.today().weekday()) % 7) + relativedelta(weeks=8), 
    # expiration: datetime = pd.Timestamp(year=(datetime.today() + relativedelta(months=5)).year, month=(datetime.today() + relativedelta(months=5)).month, day=1) + pd.offsets.MonthEnd(0) - pd.offsets.Week(weekday=4), 
    expiration: datetime = pd.Timestamp(year=(datetime.today()+relativedelta(months=5)).year, month=(datetime.today()+relativedelta(months=5)).month, day=1) + pd.offsets.Week(weekday=4) + pd.offsets.Week(2), 
    stock_name: str = 'QQQ',
    option_type: str = 'C',
):
    '''
    Return the symbol of an option, given the strike price and the expiration
    '''
    leading_zeros = 4 - int(np.log10(strike))
    ending_zeros = 8 - leading_zeros - len(str(strike*1000))
    return stock_name.upper() + expiration.strftime('%Y%m%d')[2:] + option_type + '0'*leading_zeros + str(strike*1000) + '0'*ending_zeros

In [3]:
df_qqq = yf.download('qqq'.upper(),
                     start=(datetime.today() - relativedelta(months=4)).strftime('%Y-%m-%d'),
                     prepost=True,
                     auto_adjust=True
                     ).droplevel(level='Ticker', axis=1)
df_qqq.index = pd.to_datetime(df_qqq.index)
df_qqq.columns = df_qqq.columns.str.lower()
print(df_qqq.shape)
df_qqq.head()

[*********************100%***********************]  1 of 1 completed

(87, 5)


Price,close,high,low,open,volume
Date,,,,,
2025-08-05,559.621277,565.903972,559.081863,564.735343,48666600
2025-08-06,566.663086,567.102579,559.980830,560.460255,41823700
2025-08-07,568.580872,572.656164,564.455649,571.008051,44463000
2025-08-08,573.884705,574.104482,569.489836,569.789476,35255500
2025-08-11,572.186646,576.132084,570.908159,574.024542,33112900


In [5]:
days_till_expirations = []
call_option_symbols = []
option_open = []
option_high = []
option_low = []
option_close = []
option_volume = []
# put_option_symbols = []
# put_option_open = []
# put_option_high = []
# put_option_low = []
# put_option_close = []
# put_option_volume = []

# previous_row = None 
for idx, row in df_qqq.iterrows():
    # if previous_row is not None:
        try:
            # strike = int((previous_row['close'] + 2.5) // 5 * 5)
            strike = int((row['open'] + 2.5) // 5 * 5)
            expiration = pd.Timestamp(year=(idx+relativedelta(months=4)).year, month=(idx+relativedelta(months=4)).month, day=1) + pd.offsets.Week(weekday=4) + pd.offsets.Week(2)
            call_option_symbol = option_symbol(
                strike=strike,
                expiration=expiration,
            )
            df_option = yf.download(call_option_symbol,
                    start=idx.strftime('%Y-%m-%d'),
                    prepost=True,
                    auto_adjust=True
                    ).droplevel(level='Ticker', axis=1)
            # put_option_symbol = option_symbol(
            #     strike=strike,
            #     expiration=expiration,
            #     option_type='P',
            # )
            # df_put_option = yf.download(put_option_symbol,
            #         start=idx.strftime('%Y-%m-%d'),
            #         prepost=True,
            #         auto_adjust=True
            #         ).droplevel(level='Ticker', axis=1)
            # if df_put_option[df_put_option.index==idx].shape[0] > 0 and df_option[df_option.index==idx].shape[0] > 0:
            option_open.append(df_option[df_option.index==idx]['Open'][0])
            option_high.append(df_option[df_option.index==idx]['High'][0])
            option_low.append(df_option[df_option.index==idx]['Low'][0])
            option_close.append(df_option[df_option.index==idx]['Close'][0])
            option_volume.append(df_option[df_option.index==idx]['Volume'][0])
            call_option_symbols.append(call_option_symbol)
            days_till_expirations.append((
                pd.Timestamp(year=(idx+relativedelta(months=4)).year, month=(idx+relativedelta(months=4)).month, day=1)
                +
                pd.offsets.Week(weekday=4)
                +
                pd.offsets.Week(2) - idx).days)
            # put_option_open.append(df_put_option[df_put_option.index==idx]['Open'][0])
            # put_option_high.append(df_put_option[df_put_option.index==idx]['High'][0])
            # put_option_low.append(df_put_option[df_put_option.index==idx]['Low'][0])
            # put_option_close.append(df_put_option[df_put_option.index==idx]['Close'][0])
            # put_option_volume.append(df_put_option[df_put_option.index==idx]['Volume'][0])
            # put_option_symbols.append(put_option_symbol)
            # else:
            #     df_qqq.drop(idx, inplace=True)
        except:
            df_qqq.drop(idx, inplace=True)
        # previous_row = row
    
df_qqq['call_option_symbols'] = call_option_symbols
df_qqq['option_open'] = option_open
df_qqq['option_high'] = option_high
df_qqq['option_low'] = option_low
df_qqq['option_close'] = option_close
df_qqq['option_volume'] = option_volume

# df_qqq['put_option_symbols'] = put_option_symbols
# df_qqq['put_option_open'] = put_option_open
# df_qqq['put_option_high'] = put_option_high
# df_qqq['put_option_low'] = put_option_low
# df_qqq['put_option_close'] = put_option_close
# df_qqq['put_option_volume'] = put_option_volume
# df_qqq['days_till_expiration'] = days_till_expirations

print(df_qqq.shape)
df_qqq

[*********************100%***********************]  1 of 1 completed
/var/folders/nb/2_t_pyss0gs6mwvk974rxmz80000gn/T/ipykernel_9143/3670820244.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  option_open.append(df_option[df_option.index==idx]['Open'][0])
/var/folders/nb/2_t_pyss0gs6mwvk974rxmz80000gn/T/ipykernel_9143/3670820244.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  option_high.append(df_option[df_option.index==idx]['High'][0])
/var/folders/nb/2_t_pyss0gs6mwvk974rxmz80000gn/T/ipykernel_9143/3670820244.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future ver

(66, 11)



/var/folders/nb/2_t_pyss0gs6mwvk974rxmz80000gn/T/ipykernel_9143/3670820244.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  option_open.append(df_option[df_option.index==idx]['Open'][0])
/var/folders/nb/2_t_pyss0gs6mwvk974rxmz80000gn/T/ipykernel_9143/3670820244.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  option_high.append(df_option[df_option.index==idx]['High'][0])
/var/folders/nb/2_t_pyss0gs6mwvk974rxmz80000gn/T/ipykernel_9143/3670820244.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with

Price,close,high,low,open,volume,call_option_symbols,option_open,option_high,option_low,option_close,option_volume
Date,,,,,,,,,,,
2025-08-05,559.621277,565.903972,559.081863,564.735343,48666600,QQQ251219C00565000,29.950001,29.950001,28.799999,29.440001,47
2025-08-06,566.663086,567.102579,559.980830,560.460255,41823700,QQQ251219C00560000,33.169998,35.419998,33.070000,35.419998,24
2025-08-07,568.580872,572.656164,564.455649,571.008051,44463000,QQQ251219C00570000,31.940001,32.529999,27.950001,30.510000,0
2025-08-08,573.884705,574.104482,569.489836,569.789476,35255500,QQQ251219C00570000,31.780001,33.169998,31.780001,33.169998,106
2025-08-11,572.186646,576.132084,570.908159,574.024542,33112900,QQQ251219C00575000,29.910000,30.879999,28.000000,28.000000,85
...,...,...,...,...,...,...,...,...,...,...,...
2025-12-01,617.169983,619.440002,612.520020,613.630005,40957600,QQQ260417C00615000,35.750000,38.340000,35.750000,37.430000,61
2025-12-02,622.000000,623.750000,617.590027,619.460022,54541800,QQQ260417C00620000,37.130001,37.759998,35.869999,37.250000,251
2025-12-03,623.520020,624.219971,618.030029,619.619995,47841600,QQQ260417C00620000,36.000000,38.130001,36.000000,37.990002,28


In [ ]:
# # Don't do arbitrage from put-call parity violation -- too complicated
# df_qqq['risk_free_rate_from_parity'] = 365/df_qqq['days_till_expiration']*np.log(
#     ((df_qqq['open'] + 2.5) // 5 * 5).astype(int)
#     /
#     (df_qqq['put_option_close']+df_qqq['close']-df_qqq['option_close'])
# )
# df_qqq

### Commodity?

#### 上海黄金交易所: do-able for > 20%, once per year

In [17]:
import akshare as ak
# Nothing works except 螺纹钢
# ak.spot_price_qh(symbol="螺纹钢")
df = ak.spot_hist_sge(symbol='Au99.99')
df.head()

,date,open,close,low,high
0,2016-12-19,262.45,262.76,262.02,263.50
1,2016-12-20,262.88,262.06,261.42,263.70
2,2016-12-21,262.40,260.97,258.60,262.65
3,2016-12-22,261.18,260.00,259.05,261.18
4,2016-12-23,258.95,260.07,258.70,260.37


In [39]:
((df['high'] - df['low']) / df['open']).describe(percentiles=[0.25,0.5,0.75,0.9,0.95,0.99])

count    2177.000000
mean        0.012249
std         0.017047
min         0.000000
25%         0.006180
50%         0.008948
75%         0.013372
90%         0.020749
95%         0.029065
99%         0.054549
max         0.311299
dtype: float64

In [47]:
df_sub = df[df['high'] - df['low'] >= 0.2*df['open']]
df_sub['diff'] = (df_sub['high'] - df_sub['low'])/df_sub['open']
df_sub

/var/folders/nb/2_t_pyss0gs6mwvk974rxmz80000gn/T/ipykernel_9143/651406940.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['diff'] = (df_sub['high'] - df_sub['low'])/df_sub['open']


,date,open,close,low,high,diff
691,2019-10-24,285.00,339.94,285.00,343.00,0.203509
956,2020-11-26,379.90,378.85,278.39,380.70,0.269308
1165,2021-10-12,367.00,367.37,276.99,367.79,0.247411
1220,2021-12-28,378.80,373.55,260.88,378.80,0.311299
1222,2021-12-30,360.80,369.79,280.01,371.10,0.252467
1246,2022-02-10,374.95,375.70,275.05,376.30,0.270036
2171,2025-12-01,946.20,958.46,756.40,962.20,0.217502


In [32]:
df_gld = yf.download('gc=f'.upper(),
                     start=(datetime.today() - relativedelta(years=5)).strftime('%Y-%m-%d'),
                     prepost=True,
                     auto_adjust=True
                     ).droplevel(level='Ticker', axis=1)
df_gld = df_gld.reset_index()
df_gld.head()

[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
0,2020-12-08,1870.800049,1875.800049,1861.800049,1863.000000,121
1,2020-12-09,1834.599976,1867.099976,1827.000000,1867.099976,215
2,2020-12-10,1833.599976,1849.099976,1828.000000,1833.199951,177
3,2020-12-11,1839.800049,1846.000000,1829.099976,1838.500000,109
4,2020-12-14,1828.699951,1835.699951,1822.699951,1835.500000,258


In [41]:
((df_gld['High'] - df_gld['Low']) / df_gld['Open']).describe(percentiles=[0.25,0.5,0.75,0.9,0.95,0.99])

count    1258.000000
mean        0.009938
std         0.007165
min         0.000000
25%         0.005108
50%         0.008538
75%         0.013248
90%         0.018596
95%         0.023515
99%         0.035352
max         0.056547
dtype: float64

In [36]:
df_gld[df_gld['Date']>='2025-11-28']

Price,Date,Close,High,Low,Open,Volume
1251,2025-11-28,4218.299805,4223.899902,4140.000000,4163.399902,3615
1252,2025-12-01,4239.299805,4262.100098,4214.899902,4218.500000,913
1253,2025-12-02,4186.600098,4230.500000,4167.000000,4230.500000,366
1254,2025-12-03,4199.299805,4234.100098,4195.700195,4215.700195,1897
1255,2025-12-04,4211.799805,4211.799805,4175.000000,4211.000000,616
1256,2025-12-05,4212.899902,4255.700195,4194.500000,4204.100098,616
1257,2025-12-08,4215.200195,4247.899902,4204.000000,4205.500000,119923


In [49]:
ak.futures_hist_table_em()

,市场简称,合约中文代码,合约代码
0,上期所,沪铜2512,cu2512
1,上期所,沪铜2601,cu2601
2,上期所,沪铜2602,cu2602
3,上期所,沪铜2603,cu2603
4,上期所,沪铜2604,cu2604
...,...,...,...
1046,广期所,钯2606,pd2606
1047,广期所,钯2608,pd2608
1048,广期所,钯2610,pd2610
1049,广期所,钯主连,pdm


In [50]:
ak.match_main_contract(symbol="shfe") 

FU2601
SC2601
AL2602
RU2605
ZN2601
CU2601
AU2602
RB2605
xc_qh 无主力合约
PB2601
AG2602
BU2602
HC2605
SN2601
NI2601
SP2605
NR2602
SS2602
LU2603
BC2601
AO2601
BR2602
EC2602
AD2602
OP2602
shfe主力合约获取成功


'FU2601,SC2601,AL2602,RU2605,ZN2601,CU2601,AU2602,RB2605,PB2601,AG2602,BU2602,HC2605,SN2601,NI2601,SP2605,NR2602,SS2602,LU2603,BC2601,AO2601,BR2602,EC2602,AD2602,OP2602'

### Stock

BRK-A seem to have once comparing to BRK-B

Goog vs Googl: never

In [58]:
df = yf.download('brk-a'.upper(),
                     start=(datetime.today() - relativedelta(years=5)).strftime('%Y-%m-%d'),
                     prepost=True,
                     auto_adjust=True
                     ).droplevel(level='Ticker', axis=1)
df = df.reset_index()
df['diff'] = ((df['High'] - df['Low']) / df['Open'])
print(df['diff'].describe(percentiles=[0.25,0.5,0.75,0.9,0.95,0.99]))
df.sort_values(by='diff',ascending=False)

[*********************100%***********************]  1 of 1 completed

count    1256.000000
mean        0.015973
std         0.009443
min         0.005043
25%         0.010799
50%         0.013986
75%         0.019122
90%         0.024381
95%         0.029190
99%         0.042817
max         0.189299
Name: diff, dtype: float64


Price,Date,Close,High,Low,Open,Volume,diff
875,2024-06-03,631110.0,741971.0,623520.0,625735.0,9800,0.189299
475,2022-10-27,437900.0,468795.0,415639.0,438783.0,2400,0.121144
1086,2025-04-07,737000.0,757493.0,695517.0,702098.0,1900,0.088273
1088,2025-04-09,778421.0,784890.0,727962.0,738283.0,900,0.077109
465,2022-10-13,415222.0,417860.0,393012.0,394152.0,2600,0.063042
...,...,...,...,...,...,...,...
267,2021-12-30,452700.0,455000.0,452545.0,453728.0,1600,0.005411
183,2021-08-31,429900.0,431000.0,428706.0,429405.0,1000,0.005342
86,2021-04-14,403000.0,404150.0,402000.0,402789.0,1800,0.005338
38,2021-02-03,350701.0,351086.0,349250.0,350200.0,200,0.005243


In [59]:
df = yf.download('brk-b'.upper(),
                     start=(datetime.today() - relativedelta(years=5)).strftime('%Y-%m-%d'),
                     prepost=True,
                     auto_adjust=True
                     ).droplevel(level='Ticker', axis=1)
df = df.reset_index()
df['diff'] = ((df['High'] - df['Low']) / df['Open'])
print(df['diff'].describe(percentiles=[0.25,0.5,0.75,0.9,0.95,0.99]))
df.sort_values(by='diff',ascending=False)

[*********************100%***********************]  1 of 1 completed

count    1256.000000
mean        0.014614
std         0.007231
min         0.004439
25%         0.010034
50%         0.012870
75%         0.017367
90%         0.022951
95%         0.027144
99%         0.037952
max         0.093599
Name: diff, dtype: float64


Price,Date,Close,High,Low,Open,Volume,diff
1086,2025-04-07,490.380005,505.850006,462.100006,467.420013,12749700,0.093599
1088,2025-04-09,521.409973,523.929993,485.559998,490.000000,10153700,0.078306
465,2022-10-13,275.779999,276.779999,259.850006,260.579987,5483400,0.064970
1087,2025-04-08,492.640015,516.000000,484.190002,509.029999,9470100,0.062491
1085,2025-04-04,493.540009,521.000000,492.980011,519.000000,12605000,0.053988
...,...,...,...,...,...,...,...
737,2023-11-13,350.010010,350.649994,348.809998,350.089996,2196200,0.005256
827,2024-03-25,409.920013,411.070007,408.920013,410.670013,2804200,0.005235
86,2021-04-14,267.529999,268.440002,267.149994,267.309998,3119900,0.004826
766,2023-12-26,356.829987,357.769989,356.079987,356.899994,1964400,0.004735


In [64]:
df = yf.download('NVDA'.upper(),
                     start=(datetime.today() - relativedelta(years=5)).strftime('%Y-%m-%d'),
                     prepost=True,
                     auto_adjust=True
                     ).droplevel(level='Ticker', axis=1)
df = df.reset_index()
df['diff'] = ((df['High'] - df['Low']) / df['Open'])
print(df['diff'].describe(percentiles=[0.25,0.5,0.75,0.9,0.95,0.99]))
df.sort_values(by='diff',ascending=False)

[*********************100%***********************]  1 of 1 completed

count    1256.000000
mean        0.038363
std         0.019614
min         0.009508
25%         0.024706
50%         0.034119
75%         0.046335
90%         0.062845
95%         0.075362
99%         0.108261
max         0.177672
Name: diff, dtype: float64


Price,Date,Close,High,Low,Open,Volume,diff
1088,2025-04-09,114.309258,115.079114,97.512303,98.872057,612918300,0.177672
1086,2025-04-07,97.622276,101.731531,86.604280,87.444124,611041300,0.172993
229,2021-11-04,29.739744,31.300529,27.062259,27.173030,1153631000,0.155973
305,2022-02-24,23.702087,23.753987,20.849613,20.974371,735801000,0.138473
918,2024-08-05,100.407021,103.365761,90.651202,92.020611,552842400,0.138171
...,...,...,...,...,...,...,...
1148,2025-07-08,159.982071,160.202047,158.372251,159.312148,138133000,0.011486
142,2021-07-02,20.441284,20.459495,20.242480,20.394639,342764000,0.010641
1161,2025-07-25,173.480560,174.700425,172.940628,173.590549,122316800,0.010138
1135,2025-06-17,144.103836,145.203719,143.763878,144.473805,139108000,0.009966
